In [0]:
pip install lookerapi-deprecated

In [0]:
#@title Configuration

base_url = 'https://YOURINSTANCE.looker.com:19999/api/3.0/' #@param {type:"string"}
client_id = 'bNWh2QQsrkNFcWmyKhqc' #@param {type:"string"}
client_secret = 'HzTmWCKf7MJ2cBBzqZRTRbrj' #@param {type:"string"}
model_name = 'firebase' #@param {type:"string"}

In [0]:
#Get event names and user properties
import lookerapi as looker
from pprint import pprint

# instantiate Auth API
unauthenticated_client = looker.ApiClient(base_url)
unauthenticated_authApi = looker.ApiAuthApi(unauthenticated_client)

# authenticate client
token = unauthenticated_authApi.login(client_id=client_id, client_secret=client_secret)
client = looker.ApiClient(base_url, 'Authorization', 'token ' + token.access_token)

# instantiate User API client
userApi = looker.UserApi(client)
me = userApi.me();

# instantiate Look API client
queryAPI = looker.QueryApi(client)
body = {
  "model": model_name,
  "view":"events",
  "fields":["events.event_name","events__event_params.key","events__event_params__value.type"],
  "filters":{"events.event_date":"7 days"},
  "limit":"5000"
}
event_properties = queryAPI.run_inline_query("json",body)

queryAPI = looker.QueryApi(client)
body = {
  "model": model_name,
  "view":"events",
  "fields":["events__user_properties.key","events__user_properties__value.type"],
  "filters":{"events.event_date":"7 days"},
  "limit":"5000"
}
user_properties = queryAPI.run_inline_query("json",body)

In [0]:
import ast
print("#### User Properties ####")
print("view: user_properties_generated {")
print("extension: required")
print()

for event in ast.literal_eval(user_properties)[1:]: #skip over the first one 
    key = event['events__user_properties.key']
    type = event['events__user_properties__value.type']
    
    
    print("dimension: user_properties."+key+" {")
    print("type: "+ type)
    print("sql:")
    if  type == "string":
        print("   (SELECT value.string_value")
    else:
        print("   (SELECT value.int_value")
    print("       FROM UNNEST(${user_properties})")
    print(   "       WHERE key = '"+key+"') ;;")
    print("  }")
    print()
print ("}")

In [0]:
print("#### Event Properties ####")
print("view: events_generated {")
print("extension: required")
print()
for event in ast.literal_eval(event_properties)[1:]: #skip over the first one
    event_name = event['events.event_name']
    type = event['events__event_params__value.type']
    key = event['events__event_params.key']
  
    print("dimension: "+event_name+"."+key+" {")
    print("type: "+ type)
    print("sql: CASE WHEN ${event_name} = '"+event_name+"' THEN")
    if  type == "string":
        print("   (SELECT value.string_value")
    else:
        print("   (SELECT value.int_value")
    print("       FROM UNNEST(${event_params})")
    print(   "       WHERE key = '"+key+"')")
    print(   " END ;;")
    print("  }")
    print()
print("}")

  
